<a href="https://colab.research.google.com/github/junkyuhufs/Practice/blob/main/prepro_wos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##wos에서 article만 선택 (12-10-2024)

In [ ]:
# 코랩에서 파일 업로드
from google.colab import files

uploaded = files.upload()  # 파일을 업로드하세요 (예: test2.txt)

# 파일 이름 가져오기
file_name = list(uploaded.keys())[0]

# Define a function to filter entries by DT
def filter_entries_by_dt(file_path, dt_filter):
    filtered_entries = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_entry = []
        include_entry = False
        for line in file:
            if line.strip() == "ER":  # End of record
                if include_entry:
                    filtered_entries.append(''.join(current_entry))
                current_entry = []
                include_entry = False
            current_entry.append(line)
            if line.startswith("DT "):
                include_entry = dt_filter in line
    return filtered_entries

# Filter for entries where DT == "Article"
filtered_entries = filter_entries_by_dt(file_name, "Article")

# Save the filtered entries to a new file
filtered_file_name = 'filtered_articles.txt'
with open(filtered_file_name, 'w', encoding='utf-8') as file:
    file.writelines(filtered_entries)

# 다운로드 링크 제공
files.download(filtered_file_name)


##wos에서 무명저자 삭제

In [ ]:
# 코랩에서 파일 업로드
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 이름 가져오기
input_file_name = list(uploaded.keys())[0]
output_file_name = 'cleaned_articles.txt'

# CR 섹션을 청소하는 함수 정의
def clean_cr_entries(input_file_name, output_file_name):
    cleaned_entries = []
    with open(input_file_name, 'r', encoding='utf-8') as file:
        current_entry = []
        for line in file:
            if line.strip() == "ER":  # End of record
                # Process the CR section of the current entry
                cleaned_entry = []
                inside_cr = False
                cr_lines = []
                for entry_line in current_entry:
                    if entry_line.startswith("CR "):  # Start of CR section
                        inside_cr = True
                        cr_lines = [entry_line[3:].strip()]
                    elif inside_cr and entry_line.startswith("   "):  # Continuation of CR
                        cr_lines.append(entry_line.strip())
                    elif inside_cr:  # End of CR section
                        # Filter out 'Anonymous' references and reconstruct the CR section
                        cleaned_cr_lines = [
                            line for line in cr_lines if "Anonymous" not in line
                        ]
                        if cleaned_cr_lines:
                            cleaned_entry.append("CR " + cleaned_cr_lines[0] + "\n")
                            cleaned_entry.extend(
                                ["   " + line + "\n" for line in cleaned_cr_lines[1:]]
                            )
                        inside_cr = False
                        cleaned_entry.append(entry_line)
                    else:
                        cleaned_entry.append(entry_line)
                if inside_cr:  # Handle CR section at the end of an entry
                    cleaned_cr_lines = [
                        line for line in cr_lines if "Anonymous" not in line
                    ]
                    if cleaned_cr_lines:
                        cleaned_entry.append("CR " + cleaned_cr_lines[0] + "\n")
                        cleaned_entry.extend(
                            ["   " + line + "\n" for line in cleaned_cr_lines[1:]]
                        )
                cleaned_entries.append("".join(cleaned_entry))
                current_entry = []
            else:
                current_entry.append(line)
    # Write the cleaned entries to the output file
    with open(output_file_name, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_entries)

# 파일을 처리하여 결과 생성
clean_cr_entries(input_file_name, output_file_name)

# 결과 파일 다운로드
files.download(output_file_name)
